# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [84]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [150]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [151]:
Weather_csv = "Weather_data.csv"
weather_df = pd.read_csv(Weather_csv)
weather_df


,Country,City,Latitude,Longitude,Date,Max Temperature,Min Temperature,Wind Speed,Humidity,Cloudiness
0,GB,aberdeen,57.1437,-2.0981,1639872819,3.27,0.67,1.03,100,90
1,US,albany,42.6001,-73.9662,1639872857,2.60,-1.83,0.89,92,100
2,ID,padang,-0.9492,100.3543,1639872751,24.03,24.03,2.30,91,100
3,RU,igarka,67.4667,86.5833,1639872857,-38.60,-38.60,2.02,98,36
4,RU,ossora,59.2353,163.0719,1639872858,-19.14,-19.14,3.88,81,34
...,...,...,...,...,...,...,...,...,...,...
561,PK,keti bandar,24.1447,67.4497,1639873010,16.70,16.70,4.40,21,50
562,LY,surt,31.2089,16.5887,1639873011,14.19,14.19,7.56,68,63
563,MK,struga,41.1781,20.6761,1639873011,-5.46,-5.46,3.01,88,11
564,CA,lac du bonnet,50.2536,-96.0612,1639873011,-17.73,-17.73,1.17,97,80


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [152]:
gmaps.configure(api_key=g_key)

In [164]:
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [154]:
ideal_weather = weather_df
ideal_weather = ideal_weather.loc[(weather_df["Max Temperature"]<=26) & (weather_df["Min Temperature"]>=18)]
ideal_weather = ideal_weather.loc[(weather_df["Wind Speed"]<20)]
ideal_weather = ideal_weather.loc[(weather_df["Humidity"]<60)]
ideal_weather = ideal_weather.loc[(weather_df["Cloudiness"] == 0)]
ideal_weather = ideal_weather.dropna()
ideal_weather

,Country,City,Latitude,Longitude,Date,Max Temperature,Min Temperature,Wind Speed,Humidity,Cloudiness
50,AU,carnarvon,-24.8667,113.6333,1639872871,24.04,24.04,8.75,50,0
100,AU,hobart,-42.8794,147.3294,1639872805,25.57,21.17,2.24,30,0
265,NZ,ngunguru,-35.6167,174.5000,1639872928,25.09,25.09,2.54,57,0
386,NZ,kaeo,-35.1000,173.7833,1639872962,25.18,25.18,2.96,56,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [155]:
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ""

for index, row in hotel_df.iterrows():
    location = f"{row['Latitude']},{row['Longitude']}"
    params = {"location": location,
              "radius": 5000,
              "type": 'lodging',
              "key": g_key}
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except(KeyError, IndexError):
        print("Missing Criteria. Skipping.")


In [156]:
hotel_df

,Country,City,Latitude,Longitude,Date,Max Temperature,Min Temperature,Wind Speed,Humidity,Cloudiness,Hotel Name
50,AU,carnarvon,-24.8667,113.6333,1639872871,24.04,24.04,8.75,50,0,Hospitality Carnarvon
100,AU,hobart,-42.8794,147.3294,1639872805,25.57,21.17,2.24,30,0,Mantra on Collins Hobart
265,NZ,ngunguru,-35.6167,174.5000,1639872928,25.09,25.09,2.54,57,0,Pacific Rendezvous Motel Resort
386,NZ,kaeo,-35.1000,173.7833,1639872962,25.18,25.18,2.96,56,0,Bushido Lodge


In [168]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [170]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))